In [25]:
# uncomment as necessary
#!pip3 install -r requirements.txt
#!pip3 install pylabel
# !pip3 install wandb

In [ ]:
# this must be run from the command line and the wandb key inserted
# !python3 -m wandb login

In [27]:
import json
from IPython import display
import os
from pylabel import importer
import shutil

In [4]:
annotations_location = './Dataset/annotations/instances_val2017.json'
images_location = './Dataset/val2017'

with open(annotations_location) as json_file:
    coco_val = json.load(json_file)

In [5]:
# display.Image(os.path.join('.',images_location,coco_val['images'][1500]['file_name']))

In [7]:
# careful this assumes annotations json is in same directory as images due to bug in pylabel
dataset = importer.ImportCoco("Dataset/val2017/instances_val2017.json", path_to_images="", name="val2017coco")
dataset.df.head(5)

,img_folder,img_filename,img_path,img_id,img_width,img_height,img_depth,ann_segmented,ann_bbox_xmin,ann_bbox_ymin,...,ann_iscrowd,ann_keypoints,ann_pose,ann_truncated,ann_difficult,cat_id,cat_name,cat_supercategory,split,annotated
id,,,,,,,,,,,,,,,,,,,,,
0,,000000397133.jpg,,397133,640,427,,,217.62,240.54,...,0.0,,,,,44,bottle,kitchen,,1
1,,000000397133.jpg,,397133,640,427,,,1.0,240.24,...,0.0,,,,,67,dining table,furniture,,1
2,,000000397133.jpg,,397133,640,427,,,388.66,69.92,...,0.0,,,,,1,person,person,,1
3,,000000397133.jpg,,397133,640,427,,,135.57,249.43,...,0.0,,,,,49,knife,kitchen,,1
4,,000000397133.jpg,,397133,640,427,,,31.28,344.0,...,0.0,,,,,51,bowl,kitchen,,1


In [8]:
#random state set at 42, the meaning of life
dataset.splitter.GroupShuffleSplit(train_pct=.9, test_pct=0.1, val_pct=0,random_state=42)

In [9]:
dataset.export.ExportToYoloV5(output_path='Dataset_yolo/labels',yaml_file='dataset_yolo.yaml', copy_images=True, use_splits=True)

In [28]:
# copy custom yaml file into Yolo dataset directory
shutil.copyfile('dataset_yolo.yaml', 'Dataset_yolo/dataset_yolo.yaml')

'Dataset_yolo/dataset_yolo.yaml'

In [9]:
# training
!python3 train.py --workers 4 --device cpu --epochs 20 --batch-size 4 --data Dataset_yolo/dataset_yolo.yaml --img 640 640 --cfg cfg/training/yolov7-custom.yaml --weights 'weights/yolov7.pt' --name yolov7-custom --hyp data/hyp.scratch.custom.yaml

/Users/uqdgokay/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
YOLOR 🚀 84932d7 torch 2.1.0 CPU

Namespace(weights='weights/yolov7.pt', cfg='cfg/training/yolov7-custom.yaml', data='Dataset_yolo/dataset_yolo.yaml', hyp='data/hyp.scratch.custom.yaml', epochs=20, batch_size=4, img_size=[640, 640], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, image_weights=False, device='cpu', multi_scale=False, single_cls=False, adam=False, sync_bn=False, local_rank=-1, workers=4, project='runs/train', entity=None, name='yolov7-custom', exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias='latest', freeze=[0], v5_metric=False, world_size=1, 

In [24]:
# inference
!python3 detect.py --weights weights/yolov7.pt --conf 0.25 --img-size 640 --source Dataset/val2017/000000037777.jpg

/Users/uqdgokay/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Namespace(weights=['weights/yolov7.pt'], source='Dataset/val2017/000000037777.jpg', img_size=640, conf_thres=0.25, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
YOLOR 🚀 84932d7 torch 2.1.0 CPU

<class 'str'>
<class 'pathlib.PosixPath'>
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
/Users/uqdgokay/Library/Python/3.9/lib/python/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Us